This notebook uses a GPT-2 model fine-tuned on the Spanish portion of the [Europarl corpus](https://www.statmt.org/europarl/) to interactively generate a discourse for the European Parliament.

# Set up libraries and models

In [1]:
import numpy as np
import torch

In [2]:
# Set random seed for reproducibility
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(55)

In [3]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the original tokenizer and the model fine-tuned on Europarl data
tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/gpt2-base-bne")

model = AutoModelForCausalLM.from_pretrained('gpt2_PLANTL_base_ft_europarl', local_files_only=True)

# Create text generation pipeline
writer = pipeline('text-generation',model=model, tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Define some helper functions for text generation

In [4]:
def auto_write(prompt, max_len=50, num_seqs=5):
    '''
    Uses a text-generation pipeline to generate 'num_seqs' alternative continuations of the provided prompt
    '''
    current_len = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").shape[1]
    current_text_len = len(prompt)
    texts = writer(prompt, max_length=(current_len + max_len), num_return_sequences=num_seqs, temperature=1.0, do_sample=True, top_k=50, top_p=0.95, repetition_penalty=2.0)
    generated_options = []
    for txt in texts:
        new_text = txt['generated_text'][current_text_len:]
        generated_options.append(new_text)
        print(new_text + '\n')
    return generated_options

In [5]:
def select_alternative(previous_text, current_alternatives, sel, end_word=None):
    '''
    This auxiliary function is used to select one of the proposed alternatives and incorporate it to the discourse.
    It is also possible to select a subset of the desired alternative by providing a substring to cut on
    '''
    # Select the provided alternative. Note that the function call uses 1-based indexing
    new_text = current_alternatives[sel-1]
    
    # If a substring to cut the alternative was provided, take only the text up to that point
    if end_word is not None:
        idx = new_text.find(end_word)
        new_text = new_text[: idx if idx > 0 else None]
        
    # Update the text and return the updated text and the previous text, so that we have a backup in case we make a mistake
    updated_text = previous_text + new_text
    return updated_text, previous_text

In [6]:
def add_manually(previous_text, new_text):
    '''
    Manually adds some text to the discourse
    '''
    updated_text = previous_text + new_text
    return updated_text, previous_text

# Let's write!

In [7]:
current_text = "Muchas gracias Señora Presidenta. Con su permiso, me gustaría comenzar dando las gracias"

In [8]:
generated_options = auto_write(current_text)

 a todos los presentes por sus aportaciones positivas y constructivas durante la elaboración del informe sobre el Libro Verde de esta directiva que hoy nos ocupa: es un texto extremadamente importante para garantizar una mayor competencia leal con relación al abastecimiento eléctrico en toda Europa; constituye

 a sus Señorías por este debate tan importante y estimulante sobre los derechos de la mujer en el mundo moderno: "Todo lo que he aprendido es absolutamente fundamental para nuestro futuro". Creo necesario reconocer aquí algunos principios importantes cuya importancia se refleja principalmente hoy cuando

 al señor Marinescu por haber presentado un informe tan rápido y de una importancia capital sobre la iniciativa «Todo menos armas» en el que pedimos a los diputados del Parlamento Europeo (al contrario quizás) -que usted sabe- qué papel desempeñar dentro

 al Comisario por todo el trabajo que ha hecho en relación con la propuesta de un reglamento relativo a los servicios aéreos 

In [9]:
current_text, backup_text = select_alternative(current_text, generated_options, 1, ';')

In [10]:
print(current_text)

Muchas gracias Señora Presidenta. Con su permiso, me gustaría comenzar dando las gracias a todos los presentes por sus aportaciones positivas y constructivas durante la elaboración del informe sobre el Libro Verde de esta directiva que hoy nos ocupa: es un texto extremadamente importante para garantizar una mayor competencia leal con relación al abastecimiento eléctrico en toda Europa


In [11]:
generated_options = auto_write(current_text)

 -en mi opinión- tras cuatro décadas; especialmente teniendo claro también lo fundamental acerca nuestra política energética común dentro nuestros socios exteriores como son Rusia e Irán (que han desempeñado realmente uno papel clave) o países productores estratégicos importantes fuera pero muy dependientes actualmente tan

; no solo eso sino también hay propuestas como ésta presentadas desde hace mucho tiempo destinadas especialmente respecto este sistema energético integrado lo cual contribuye notablemente tanto a mejorar nuestro suministro de electricidad (la propuesta inicial iba encaminada fundamentalmente precisamente aquí hacia hacer más estrictas exigencias) o

 mediante nuevas tecnologías; implica mejorar tanto aspectos tecnológicos como notecnológicos desde nuestro punto vista técnico o comercial –por ejemplo aquí, yo diría– debido también especialmente nuestros respectivos mercados públicos europeos -esto ya hay mucha gente muy poca cualificada-. El ponente ha

 -de hecho so

In [12]:
current_text, backup_text = select_alternative(current_text, generated_options, 5, '?')

In [13]:
current_text, backup_text = add_manually(current_text, '? En el último año, los ciudadanos europeos han experimentado \
una subida de los precios de la electricidad que resulta insoportable.')

In [14]:
print(current_text)

Muchas gracias Señora Presidenta. Con su permiso, me gustaría comenzar dando las gracias a todos los presentes por sus aportaciones positivas y constructivas durante la elaboración del informe sobre el Libro Verde de esta directiva que hoy nos ocupa: es un texto extremadamente importante para garantizar una mayor competencia leal con relación al abastecimiento eléctrico en toda Europa; supone establecer unos límites superiores comunes entre productos distintos -en particular nuevos competidores como Japón- pero también normas mínimas aplicables más allá donde haya escasez geográfica o tecnológica imperante desde hace muchos años ¿Cómo definiría mejor lo bien conocido? En el último año, los ciudadanos europeos han experimentado una subida de los precios de la electricidad que resulta insoportable.


In [15]:
generated_options = auto_write(current_text)

 Me satisface oírle afirmarlo claramente cuando se trata únicamente de algunos indicadores económicos muy importantes tanto nacionales, tal vez no sea posible predecir cuál va siendo cada día esa escalada concreta dentro nuestra Unión Europea, especialmente aquí mismo hasta este momento histórico antes ni cuándo

 El impacto medioambiental se ha detenido mientras tanto debido no solo nuestra propia atmósfera sino algunos sectores industriales particulares muy sensibles tales cómo grandes compañías petroleras internacionales e importantes suministradores domésticos como China llevan ya mucho tiempo pidiendo unas tarifas reducidas sin margen competitivo bajo ningún concepto

 Esta situación merece ser resaltada especialmente porque va acompañada igualmente clara e indiscutiblemente contraria tanto fuera del país vecino oriental (Rusia) cómo dentro Rusia; se puede constatar además otra forma aún mucho menos evidente de evolución económica relacionada directa no solo respecto –como

 Su pr

In [16]:
text_v1 = current_text

In [17]:
current_text = "En el último año, los ciudadanos europeos han experimentado una subida de los precios de la electricidad que resulta insoportable."

In [18]:
generated_options = auto_write(current_text)

 Las compañías tienen problemas para encontrar a quienes les pagarán este servicio por adelantado y tendrán más dificultades en las redes eléctricas durante un período transitorio limitado -un tiempo considerable del cual deberían ser libres unos cuantos años- si no pueden utilizar su energía directamente con fuentes

 Las expectativas económicas para las energías renovables se sitúan en promedio tres veces más alto y cinco al día entre 2000 y 2008; además existe un déficit energético enorme -de 400 millones a 50 000 toneladas- debido principalmente ante todo por no haberse tomado medidas

 Por eso hay más gente en situación precaria y están menos protegidos frente a las subidas injustas del precio único eléctrico o gas natural con un nivel insostenible para sus familias e intereses particulares? ¿Vamos entonces hasta Europa Central; qué hemos hecho durante años al respecto

 Es comprensible por qué se trata simplemente a las personas mayores como pensionistas en un país con muy pocos 

In [19]:
current_text, backup_text = select_alternative(current_text, generated_options, 3, ' o gas')

In [20]:
current_text, backup_text = add_manually(current_text, '.')

In [21]:
generated_options = auto_write(current_text)

 La consecuencia es un aumento espectacular del coste económico al tiempo social para estos sectores vulnerables: por ejemplo trabajadores móviles o discapacitados físicos; grupos enteros -como pueden ser alumnos universitarios- son objeto frecuente e irreversible también porque no tienen cobertura eléctrica fija con tarifas únicas

 ¿Cuántos hogares se verán afectados esta noche por estas fluctuaciones?  Muchas personas podrían pensar sobre esto después o quizás no lo creerán ahora mismo; pero entonces será así: algunos sencillamente dirán "no entiendo muy bien". No tengo ninguna objeción

 Una sociedad europea competitiva tiene por lo tanto un gran interés para mantener al día esta oferta atractiva e incrementar así su competitividad futura desde dentro hacia fuera". El objetivo final es proteger ese sector financiero también mediante unas nuevas reglas económicas transparentes destinadas esencialmente contraataques

 Y no es ninguna casualidad pues hemos sabido -y así va mi país- si

In [22]:
current_text, backup_text = select_alternative(current_text, generated_options, 5, ' Las fuentes')

In [23]:
current_text, backup_text = add_manually(current_text, ' Señorías, debemos')

In [24]:
generated_options = auto_write(current_text)

 apoyar firmemente estos esfuerzos con dinero público –no queremos financiarlos aquí ni tampoco financiarlos– pero no podemos renunciar ante tales hechos consumados: reducir nuestros suministros energéticos dentro nuestras fronteras actualmente sin tener recursos disponibles bajo gestión estatal también supone ayudar mucho; algo

 asegurarnos también con frecuencia -al igual que nosotros- entre nuestros proveedores principales –los Estados Unidos– pero no sin dificultades puesto que es precisamente gracias estas normas legislativas transnacionales europeas disponemos ahora mucho mejor desde hoy cuando se cumplen nuestras expectativas si nos aseguramos nuestra

 tomar esta decisión política sin diluirla: deberíamos empezar con tiempo suficiente antes incluso de lo inicialmente previsto -lo cual implica necesariamente tener mucho cuidado-, pero también porque no podemos imponer restricciones significativas si se requiere algún tipo incentivo adicional hasta 2013; primero debe

 mejorar n

In [25]:
current_text, backup_text = select_alternative(current_text, generated_options, 3, ':')

In [26]:
current_text, backup_text = add_manually(current_text, '. Me gustaría concluir mi intervención agradeciendo nuevamente')

In [27]:
generated_options = auto_write(current_text)

 no solo lo mucho ya logrado hasta ahora sino también mis esfuerzos constantes; esto es fundamental porque me permite estar hoy aquí con ustedes: les agradezco sinceramente nuestra cooperación parlamentaria tan constructiva siempre presente -siempre basada realmenteen usted, señor Wynn-. Nos

 públicamente dicha ayuda hacia usted realmente si ha tenido presente lo mucho importante -estoy hablando aquí hoy- así como pedirle encarecidamente toda nuestra atención respecto a sus esfuerzos incansables cuando fue necesario hace siete años ya desde 1993 para llegar hasta nuestro punto

 este compromiso con ustedes, también porque no siempre existe algo parecido entre nosotros pero podemos hacerlo muy bien cuando lo hacemos juntos desde hace mucho tiempo aquí: deberíamos decir "sí" hoy; votemos ahora contra cualquier nueva legislación acerca -esta vez es peor

 sus contribuciones muy útiles hacia este debate anual con ustedes porque les aseguro sinceramente -y considero especialmente significa

In [28]:
current_text, backup_text = select_alternative(current_text, generated_options, 5, ' porque')

In [29]:
current_text, backup_text = add_manually(current_text, '.\nMuchas gracias.')

In [30]:
final_text = text_v1 + current_text
print(final_text)

Muchas gracias Señora Presidenta. Con su permiso, me gustaría comenzar dando las gracias a todos los presentes por sus aportaciones positivas y constructivas durante la elaboración del informe sobre el Libro Verde de esta directiva que hoy nos ocupa: es un texto extremadamente importante para garantizar una mayor competencia leal con relación al abastecimiento eléctrico en toda Europa; supone establecer unos límites superiores comunes entre productos distintos -en particular nuevos competidores como Japón- pero también normas mínimas aplicables más allá donde haya escasez geográfica o tecnológica imperante desde hace muchos años ¿Cómo definiría mejor lo bien conocido? En el último año, los ciudadanos europeos han experimentado una subida de los precios de la electricidad que resulta insoportable.En el último año, los ciudadanos europeos han experimentado una subida de los precios de la electricidad que resulta insoportable. Por eso hay más gente en situación precaria y están menos prot